# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'01-21-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'01-21-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-01-22 05:22:10,33.93911,67.709953,54403,2363,46887,5153.0,Afghanistan,139.751653,4.343510
1,NaN,NaN,NaN,Albania,2021-01-22 05:22:10,41.15330,20.168300,69916,1296,42426,26194.0,Albania,2429.494753,1.853653
2,NaN,NaN,NaN,Algeria,2021-01-22 05:22:10,28.03390,1.659600,104852,2853,71343,30656.0,Algeria,239.109478,2.720978
3,NaN,NaN,NaN,Andorra,2021-01-22 05:22:10,42.50630,1.521800,9379,93,8474,812.0,Andorra,12138.743286,0.991577
4,NaN,NaN,NaN,Angola,2021-01-22 05:22:10,-11.20270,17.873900,19177,448,17176,1553.0,Angola,58.348578,2.336132


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,53584,53584,53775,53831,53938,53984,54062,54141,54278,54403
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,64627,65334,65994,66635,67216,67690,67982,68568,69238,69916
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,102641,102860,103127,103381,103611,103833,104092,104341,104606,104852


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2301,2301,2314,2324,2336,2339,2343,2346,2354,2363
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,1252,1256,1261,1265,1270,1277,1281,1287,1291,1296
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2816,2819,2822,2827,2831,2836,2840,2843,2849,2853


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,44608,44850,45298,45434,45465,45868,46359,46554,46759,46887
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,38421,38860,39246,39625,40090,40453,40870,41464,41969,42426
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,69608,69791,69992,70188,70373,70554,70747,70933,71127,71343


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,4970,4998,5075,5103,5154,5184,5198,5227,5257,5270
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,15572,15701,15841,16002,16176,16251,16346,16513,16653,16798
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1679,1685,1696,1712,1723,1729,1730,1738,1760,1778


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,55,55,55,55,55,55,55,55,55,61
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,175,175,177,179,182,182,182,183,184,188
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,35,35,36,36,36,36,36,36,36,36


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
651,1001.0,Autauga,Alabama,US,2021-01-22 05:22:10,32.539527,-86.644082,5270,61,0,5209.0,"Autauga, Alabama, US",9432.780254,1.157495
688,1075.0,Lamar,Alabama,US,2021-01-22 05:22:10,33.779950,-88.096680,1196,21,0,1175.0,"Lamar, Alabama, US",8663.527707,1.755853
689,1077.0,Lauderdale,Alabama,US,2021-01-22 05:22:10,34.901719,-87.656247,7733,99,0,7634.0,"Lauderdale, Alabama, US",8339.354463,1.280228


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,24627675,410102,0
India,10625424,153030,10283702
Brazil,8697368,214147,7673092


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,24627675,410102,0,24217573,2021-01-22 05:22:10,37.936303,-91.379001
India,10625424,153030,10283702,188692,2021-01-22 05:22:10,23.088275,81.806127
Brazil,8697368,214147,7673092,810129,2021-01-22 05:22:10,-12.669522,-48.480493
Russia,3616680,66810,3021861,528009,2021-01-22 05:22:10,54.546312,62.120860
United Kingdom,3553773,94765,8581,3450427,2021-01-22 05:22:10,30.744028,-35.367255


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3095814,35549,0
Texas,2208871,33942,0
Florida,1613884,24739,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3095814,35549,0,3060265,2021-01-22 05:22:10,37.843962,-120.728594
Texas,2208871,33942,0,2174929,2021-01-22 05:22:10,31.660643,-98.653069
Florida,1613884,24739,0,1589145,2021-01-22 05:22:10,28.940755,-82.700744
New York,1293719,41787,0,1251932,2021-01-22 05:22:10,42.544151,-75.474183
Illinois,1086333,20423,0,1065910,2021-01-22 05:22:10,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1046021,14641,0
Illinois,Cook,439118,9148,0
Arizona,Maricopa,434337,6683,0
Florida,Miami-Dade,352405,4644,0
Texas,Harris,293271,3898,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1046021,14641,0,1031380,2021-01-22 05:22:10,34.308284,-118.228241,6037.0
Illinois,Cook,439118,9148,0,429970,2021-01-22 05:22:10,41.841448,-87.816588,17031.0
Arizona,Maricopa,434337,6683,0,427654,2021-01-22 05:22:10,33.348359,-112.491815,4013.0
Florida,Miami-Dade,352405,4644,0,347761,2021-01-22 05:22:10,25.611236,-80.551706,12086.0
Texas,Harris,293271,3898,0,289373,2021-01-22 05:22:10,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-19,54141,68568,104341,9194,19011,190,1819569,164912,28740,396425,...,3476804,33446,78091,1,121117,1540,153093,2115,39515,28675
2021-01-20,54278,69238,104606,9308,19093,190,1831681,165221,28749,398096,...,3515796,34294,78163,1,121691,1544,153590,2115,40949,29408
2021-01-21,54403,69916,104852,9379,19177,192,1843077,165528,28755,399798,...,3553773,34992,78219,1,122260,1546,154063,2115,42213,30047


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-19,2346,1287,2843,92,442,6,46066,3007,909,7199,...,91643,330,620,0,1116,35,1741,612,578,825
2021-01-20,2354,1291,2849,92,444,6,46216,3016,909,7237,...,93469,336,620,0,1122,35,1751,612,585,879
2021-01-21,2363,1296,2853,93,448,6,46355,3021,909,7288,...,94765,347,620,0,1129,35,1757,614,597,917


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-19,46554,41464,70933,8349,16822,157,1604373,153500,25943,373090,...,8363,25410,76585,1,113652,1402,140914,1422,28066,18110
2021-01-20,46759,41969,71127,8399,16921,157,1613773,153857,25952,374824,...,8492,26118,76624,1,113982,1406,142004,1423,29775,19253
2021-01-21,46887,42426,71343,8474,17176,162,1625755,154046,25961,376360,...,8581,26927,76655,1,114299,1411,142819,1423,31522,19569


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,4970,4998,5075,5103,5154,5184,5198,5227,5257,5270
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1153,1157,1168,1166,1175,1180,1182,1184,1190,1196
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,7296,7350,7402,7451,7528,7546,7600,7631,7686,7733
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,2351,2368,2393,2404,2425,2436,2442,2452,2466,2491
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,11812,11983,12120,12261,12393,12465,12515,12603,12749,12901


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-19,426543,52222,0,685699,273594,3055568,377856,232219,71311,49,...,690065,2157459,326221,10321,2283,451076,291989,110820,571268,49922
2021-01-20,429655,52393,0,690544,276114,3074812,379227,234134,71775,49,...,694291,2185554,328380,10471,2305,455591,294017,111677,573119,50124
2021-01-21,432536,52605,0,699942,279220,3095814,381210,235796,72523,49,...,697783,2208871,330469,10580,2305,459604,296087,112617,575688,50424


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-01-19        5227   16513    1738  2170   5320     997   1707   11206   
2021-01-20        5257   16653    1760  2188   5376    1011   1708   11292   
2021-01-21        5270   16798    1778  2198   5411    1014   1713   11365   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-01-19         2886     1609  ...    2469    538     2802      634   
2021-01-20         2931     1619  ...    2487    540     2807      636   
2021-01-21         2973     1625  ...    2508    542     2811      639   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-01-19           3329  2769  1843          0      846    606  
2021-01-20           3347  2788  1852          0      846    606  
2021-01-21           3370  2847  1874          0      848    609  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-19,6126,230,0,11266,4386,34441,5388,6682,1021,0,...,8471,33032,1507,163,24,5798,3940,1815,5973,550
2021-01-20,6283,253,0,11528,4441,35060,5422,6726,1026,0,...,8556,33491,1517,165,24,5861,3940,1836,6035,550
2021-01-21,6379,254,0,11772,4496,35549,5440,6774,1027,0,...,8684,33942,1547,168,24,5940,4065,1849,6090,550


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-01-19          55     183      36   47     83      28     58     200   
2021-01-20          55     184      36   47     83      28     58     201   
2021-01-21          61     188      36   48     86      28     58     205   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-01-19           69       30  ...      17     10       18        7   
2021-01-20           70       30  ...      17     10       18        7   
2021-01-21           70       30  ...      17     10       18        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-01-19             28     5    10          0       24      4  
2021-01-20             28     5    10          0       24      4  
2021-01-21             28     5    10          0       24      4  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-19,0.001461,0.008620,0.002392,0.012221,0.004491,0.005291,0.006717,0.001433,0.000313,0.003763,...,0.009716,0.017740,0.000705,0.0,0.005588,0.000650,0.003527,0.000947,0.034235,0.028073
2021-01-20,0.002530,0.009771,0.002540,0.012399,0.004313,0.000000,0.006657,0.001874,0.000313,0.004215,...,0.011215,0.025354,0.000922,0.0,0.004739,0.002597,0.003246,0.000000,0.036290,0.025562
2021-01-21,0.002303,0.009792,0.002352,0.007628,0.004400,0.010526,0.006222,0.001858,0.000209,0.004275,...,0.010802,0.020353,0.000716,0.0,0.004676,0.001295,0.003080,0.000000,0.030868,0.021729


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-19,0.001280,0.004684,0.001056,0.010989,0.006834,0.0,0.005106,0.003002,0.0,0.010812,...,0.017905,0.034483,0.001616,NaN,0.003597,0.0,0.008691,0.000000,0.033989,0.067270
2021-01-20,0.003410,0.003108,0.002110,0.000000,0.004525,0.0,0.003256,0.002993,0.0,0.005279,...,0.019925,0.018182,0.000000,NaN,0.005376,0.0,0.005744,0.000000,0.012111,0.065455
2021-01-21,0.003823,0.003873,0.001404,0.010870,0.009009,0.0,0.003008,0.001658,0.0,0.007047,...,0.013866,0.032738,0.000000,NaN,0.006239,0.0,0.003427,0.003268,0.020513,0.043231


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-19,0.004206,0.014534,0.002629,0.023915,0.008695,0.006410,0.006023,0.002848,0.000463,0.005999,...,0.014189,0.032172,0.000614,0.0,0.002859,0.000000,0.005990,0.000704,0.027043,0.042482
2021-01-20,0.004403,0.012179,0.002735,0.005989,0.005885,0.000000,0.005859,0.002326,0.000347,0.004648,...,0.015425,0.027863,0.000509,0.0,0.002904,0.002853,0.007735,0.000703,0.060892,0.063114
2021-01-21,0.002737,0.010889,0.003037,0.008930,0.015070,0.031847,0.007425,0.001228,0.000347,0.004098,...,0.010480,0.030975,0.000405,0.0,0.002781,0.003556,0.005739,0.000000,0.058673,0.016413


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-19,0.005931,0.002476,NaN,0.009447,0.004889,0.011859,0.004479,0.009099,0.005655,0.0,...,0.003365,0.009012,0.004007,0.009883,0.010177,0.010135,0.007070,0.009207,0.003395,0.004305
2021-01-20,0.007296,0.003274,NaN,0.007066,0.009211,0.006298,0.003628,0.008247,0.006507,0.0,...,0.006124,0.013022,0.006618,0.014533,0.009636,0.010009,0.006945,0.007733,0.003240,0.004046
2021-01-21,0.006705,0.004046,NaN,0.013610,0.011249,0.006830,0.005229,0.007098,0.010421,0.0,...,0.005030,0.010669,0.006362,0.010410,0.000000,0.008808,0.007040,0.008417,0.004482,0.005985


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-01-19      0.005579  0.010217  0.004624  0.003700  0.002261  0.006054   
2021-01-20      0.005739  0.008478  0.012658  0.008295  0.010526  0.014042   
2021-01-21      0.002473  0.008707  0.010227  0.004570  0.006510  0.002967   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-01-19      0.002938  0.004032  0.015482  0.002492  ...  0.002843   
2021-01-20      0.000586  0.007674  0.015593  0.006215  ...  0.007290   
2021-01-21      0.002927  0.006465  0.014330  0.003706  ...  0.008444   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-01-19      0.005607  0.000000  0.000000   0.005133  0.005812  0.005456   
2021-01-20      0.003717  0.001784  0.003155   0.005407  0.006862  0.004883   
2021-01-21      0.003704  0.001425  0.004717   0.006872  0.021162  0.011879   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-01-19            NaN  0.002370  0.006645  
2021-01-20            NaN  0.000000  0.000000  
2021-01-21            NaN  0.002364  0.004950  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-19,0.000817,0.004367,NaN,0.000089,0.009901,0.020595,0.000371,0.001799,0.004921,NaN,...,0.004864,0.009813,0.004667,0.000000,0.0,0.010281,0.009480,0.017377,0.007931,0.05364
2021-01-20,0.025628,0.100000,NaN,0.023256,0.012540,0.017973,0.006310,0.006585,0.004897,NaN,...,0.010034,0.013896,0.006636,0.012270,0.0,0.010866,0.000000,0.011570,0.010380,0.00000
2021-01-21,0.015279,0.003953,NaN,0.021166,0.012385,0.013948,0.003320,0.007136,0.000975,NaN,...,0.014960,0.013466,0.019776,0.018182,0.0,0.013479,0.031726,0.007081,0.009114,0.00000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                       \
Admin2           Autauga   Baldwin Barbour      Bibb    Blount Bullock Butler   
2021-01-19      0.000000  0.005495     0.0  0.000000  0.000000     0.0    0.0   
2021-01-20      0.000000  0.005464     0.0  0.000000  0.000000     0.0    0.0   
2021-01-21      0.109091  0.021739     0.0  0.021277  0.036145     0.0    0.0   

Province_State                             ... Wyoming                      \
Admin2         Calhoun  Chambers Cherokee  ...    Park    Platte  Sheridan   
2021-01-19      0.0000  0.000000      0.0  ...  0.0625  0.111111  0.058824   
2021-01-20      0.0050  0.014493      0.0  ...  0.0000  0.000000  0.000000   
2021-01-21      0.0199  0.000000      0.0  ...  0.0000  0.000000  0.000000   

Province_State                                                                  
Admin2         Sublette Sweetwater Teton     Uinta Unassigned  Washakie Weston  
2021-01-19          0.0   0.076923  0.25  0.111111        NaN  0.043478    0.0  
2021-01-20          0.0   0.000000  0.00  0.000000        NaN  0.000000    0.0  
2021-01-21          0.0   0.000000  0.00  0.000000        NaN  0.000000    0.0  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-19,0.001420,0.007432,0.002380,0.007915,0.004279,0.005765,0.005761,0.001377,0.000376,0.003635,...,0.010846,0.019143,0.000695,0.0,0.005143,0.000917,0.003475,0.000656,0.032619,0.026017
2021-01-20,0.001975,0.008602,0.002460,0.010157,0.004296,0.002883,0.006209,0.001625,0.000344,0.003925,...,0.011030,0.022249,0.000809,0.0,0.004941,0.001757,0.003361,0.000328,0.034454,0.025789
2021-01-21,0.002139,0.009197,0.002406,0.008893,0.004348,0.006704,0.006215,0.001742,0.000277,0.004100,...,0.010916,0.021301,0.000763,0.0,0.004809,0.001526,0.003220,0.000164,0.032661,0.023759


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-19,0.001814,0.004283,0.001283,0.007947,0.007360,0.003125,0.005468,0.002712,6.564388e-11,0.008459,...,0.013460,0.032531,0.000873,NaN,0.004383,5.706755e-44,0.008005,0.000005,0.027650,0.065355
2021-01-20,0.002612,0.003695,0.001697,0.003974,0.005942,0.001563,0.004362,0.002852,3.282194e-11,0.006869,...,0.016693,0.025356,0.000437,NaN,0.004880,2.853378e-44,0.006875,0.000002,0.019881,0.065405
2021-01-21,0.003218,0.003784,0.001550,0.007422,0.007476,0.000781,0.003685,0.002255,1.641097e-11,0.006958,...,0.015279,0.029047,0.000218,NaN,0.005559,1.426689e-44,0.005151,0.001635,0.020197,0.054318


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-19,0.006283,0.012337,0.002660,0.012926,0.011802,0.003625,0.006537,0.002379,0.000434,0.005471,...,0.018739,0.031919,0.000755,0.0,0.002942,0.004285,0.006274,0.000734,0.035695,0.044185
2021-01-20,0.005343,0.012258,0.002698,0.009457,0.008843,0.001812,0.006198,0.002352,0.000390,0.005059,...,0.017082,0.029891,0.000632,0.0,0.002923,0.003569,0.007005,0.000719,0.048293,0.053650
2021-01-21,0.004040,0.011574,0.002867,0.009193,0.011957,0.016830,0.006811,0.001790,0.000369,0.004579,...,0.013781,0.030433,0.000518,0.0,0.002852,0.003563,0.006372,0.000359,0.053483,0.035031


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-19,0.005338,0.003269,NaN,0.009668,0.005188,0.011294,0.004262,0.012655,0.007942,3.264025e-94,...,0.004318,0.007993,0.004445,0.013053,0.010560,0.013816,0.005804,0.009270,0.003214,0.005243
2021-01-20,0.006317,0.003272,NaN,0.008367,0.007199,0.008796,0.003945,0.010451,0.007224,1.632012e-94,...,0.005221,0.010508,0.005532,0.013793,0.010098,0.011913,0.006375,0.008501,0.003227,0.004645
2021-01-21,0.006511,0.003659,NaN,0.010988,0.009224,0.007813,0.004587,0.008775,0.008823,8.160062e-95,...,0.005125,0.010588,0.005947,0.012102,0.005049,0.010361,0.006708,0.008459,0.003855,0.005315


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-01-19      0.005360  0.008421  0.003778  0.004385  0.002466  0.004496   
2021-01-20      0.005550  0.008450  0.008218  0.006340  0.006496  0.009269   
2021-01-21      0.004011  0.008578  0.009223  0.005455  0.006503  0.006118   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-01-19      0.003845  0.004576  0.010940  0.002436  ...  0.006897   
2021-01-20      0.002215  0.006125  0.013266  0.004326  ...  0.007093   
2021-01-21      0.002571  0.006295  0.013798  0.004016  ...  0.007769   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-01-19      0.010438  0.003062  0.000259   0.005296  0.013832  0.005980   
2021-01-20      0.007078  0.002423  0.001707   0.005351  0.010347  0.005432   
2021-01-21      0.005391  0.001924  0.003212   0.006112  0.015754  0.008655   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-01-19           -1.0  0.002147  0.004285  
2021-01-20           -1.0  0.001074  0.002142  
2021-01-21           -1.0  0.001719  0.003546  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-19,0.002798,0.002516,NaN,0.002497,0.008384,0.014068,0.001468,0.004174,0.003653,NaN,...,0.005174,0.007841,0.005345,0.000645,6.634256e-07,0.007190,0.005316,0.013062,0.006774,0.027092
2021-01-20,0.014213,0.051258,NaN,0.012876,0.010462,0.016020,0.003889,0.005380,0.004275,NaN,...,0.007604,0.010869,0.005991,0.006458,3.317128e-07,0.009028,0.002658,0.012316,0.008577,0.013546
2021-01-21,0.014746,0.027605,NaN,0.017021,0.011423,0.014984,0.003605,0.006258,0.002625,NaN,...,0.011282,0.012167,0.012883,0.012320,1.658564e-07,0.011253,0.017192,0.009698,0.008845,0.006773


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-01-19      0.000084  0.004378  0.000450 -0.000323  0.001408  0.000067   
2021-01-20      0.000042  0.004921  0.000225 -0.000161  0.000704  0.000033   
2021-01-21      0.054566  0.013330  0.000113  0.010558  0.018424  0.000017   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-01-19      0.000407  0.001017  0.000757  0.001351  ...  0.031558   
2021-01-20      0.000203  0.003008  0.007625  0.000676  ...  0.015779   
2021-01-21      0.000102  0.011454  0.003812  0.000338  ...  0.007890   

Province_State                                                              \
Admin2            Platte  Sheridan  Sublette Sweetwater    Teton     Uinta   
2021-01-19      0.055559  0.029412  0.000005   0.038825  0.12500  0.056061   
2021-01-20      0.027780  0.014706  0.000003   0.019412  0.06250  0.028031   
2021-01-21      0.013890  0.007353  0.000001   0.009706  0.03125  0.014015   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-01-19      -0.501961  0.022115  0.000056  
2021-01-20      -0.501961  0.011057  0.000028  
2021-01-21      -0.501961  0.005529  0.000014  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210121,AK,50941,NaN,1380026.0,NaN,totalTestsViral,1430967,58.0,1177.0,...,1430967,1,7,86aafafe0b00c7f3e6560a03504d84b8c78c6160,0,0,0,0,0,NaN
1,20210121,AL,432536,89554.0,1715242.0,NaN,totalTestsPeopleViral,2058224,2478.0,40277.0,...,2147778,96,349,5f92e66211408aa406eeac8c5a2a8df6ea781dff,0,0,0,0,0,NaN
2,20210121,AR,279220,54977.0,2091292.0,NaN,totalTestsViral,2315535,1160.0,13055.0,...,2370512,55,73,006e2c2bc783934f97cd193ff6d289ce2839dbab,0,0,0,0,0,NaN
3,20210121,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,2140,0,0,29189567fcff71a3b768e7574f0c4251e04e11fd,0,0,0,0,0,NaN
4,20210121,AZ,699942,40988.0,2628346.0,NaN,totalTestsViral,6215572,4580.0,48604.0,...,3328288,244,397,4d8f854194b7981e0aa7d7b16602845a89b33196,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210121,AK,50941.0,NaN,1380026.0,NaN,totalTestsViral,1430967.0,58.0,1177.0,...,1430967,1,7,86aafafe0b00c7f3e6560a03504d84b8c78c6160,0,0,0,0,0,NaN
1,20210121,AL,432536.0,89554.0,1715242.0,NaN,totalTestsPeopleViral,2058224.0,2478.0,40277.0,...,2147778,96,349,5f92e66211408aa406eeac8c5a2a8df6ea781dff,0,0,0,0,0,NaN
2,20210121,AR,279220.0,54977.0,2091292.0,NaN,totalTestsViral,2315535.0,1160.0,13055.0,...,2370512,55,73,006e2c2bc783934f97cd193ff6d289ce2839dbab,0,0,0,0,0,NaN
3,20210121,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,2140,0,0,29189567fcff71a3b768e7574f0c4251e04e11fd,0,0,0,0,0,NaN
4,20210121,AZ,699942.0,40988.0,2628346.0,NaN,totalTestsViral,6215572.0,4580.0,48604.0,...,3328288,244,397,4d8f854194b7981e0aa7d7b16602845a89b33196,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-01-21,AK,50941,NaN,1380026.0,NaN,totalTestsViral,1430967,58.0,1177.0,NaN,...,1430967,1,7,86aafafe0b00c7f3e6560a03504d84b8c78c6160,0,0,0,0,0,NaN
2021-01-21,AL,432536,89554.0,1715242.0,NaN,totalTestsPeopleViral,2058224,2478.0,40277.0,NaN,...,2147778,96,349,5f92e66211408aa406eeac8c5a2a8df6ea781dff,0,0,0,0,0,NaN
2021-01-21,AR,279220,54977.0,2091292.0,NaN,totalTestsViral,2315535,1160.0,13055.0,375.0,...,2370512,55,73,006e2c2bc783934f97cd193ff6d289ce2839dbab,0,0,0,0,0,NaN
2021-01-21,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,2140,0,0,29189567fcff71a3b768e7574f0c4251e04e11fd,0,0,0,0,0,NaN
2021-01-21,AZ,699942,40988.0,2628346.0,NaN,totalTestsViral,6215572,4580.0,48604.0,1058.0,...,3328288,244,397,4d8f854194b7981e0aa7d7b16602845a89b33196,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-01-21,AK,50941.0,NaN,1380026.0,NaN,totalTestsViral,1430967.0,58.0,1177.0,NaN,...,1430967,1,7,86aafafe0b00c7f3e6560a03504d84b8c78c6160,0,0,0,0,0,NaN
2021-01-21,AL,432536.0,89554.0,1715242.0,NaN,totalTestsPeopleViral,2058224.0,2478.0,40277.0,NaN,...,2147778,96,349,5f92e66211408aa406eeac8c5a2a8df6ea781dff,0,0,0,0,0,NaN
2021-01-21,AR,279220.0,54977.0,2091292.0,NaN,totalTestsViral,2315535.0,1160.0,13055.0,375.0,...,2370512,55,73,006e2c2bc783934f97cd193ff6d289ce2839dbab,0,0,0,0,0,NaN
2021-01-21,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,2140,0,0,29189567fcff71a3b768e7574f0c4251e04e11fd,0,0,0,0,0,NaN
2021-01-21,AZ,699942.0,40988.0,2628346.0,NaN,totalTestsViral,6215572.0,4580.0,48604.0,1058.0,...,3328288,244,397,4d8f854194b7981e0aa7d7b16602845a89b33196,0,0,0,0,0,NaN
